Bottom line detection definitiva

In [43]:
import cv2
from pathlib import Path
import numpy as np
import math
import pandas as pd

Funzioni

Funzione che converte le linee da 2 punti a rho,theta

In [44]:
"""
Sructure of lines_p:
(x1, y1, x2, y2) are the coordinates of the two end points of the line segment
I want to convert them to the form rho,theta in order to easly detect the horizontal lines
"""


def convert_to_rho_theta(x1, y1, x2, y2):
    """
    Convert a line from (x1, y1, x2, y2) format to (rho, theta).
    """
    # Compute the line angle (in radians) and find the normal angle
    theta = np.arctan2(y2 - y1, x2 - x1) + np.pi / 2  # Normal angle

    # Compute rho using the normal equation
    rho = x1 * np.cos(theta) + y1 * np.sin(theta)

    # Ensure rho is positive by adjusting theta accordingly
    if rho < 0:
        rho = -rho
        theta += np.pi  # Shift by 180 degrees to maintain equivalence

    return rho, theta

Get the edges

In [45]:
def get_edges(frame):
    # Define the range for light brown color in HSV
    lower_brown = np.array([00, 30, 100])
    upper_brown = np.array([20, 200, 255])

    # Define the range for rose color in HSV
    lower_rose = np.array([150, 30, 200])
    upper_rose = np.array([180, 200, 255])

    # Convert the image to HSV color space
    hsv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create masks for brown and rose colors
    mask_brown = cv2.inRange(hsv_image, lower_brown, upper_brown)
    mask_rose = cv2.inRange(hsv_image, lower_rose, upper_rose)

    # Combine the masks
    combined_mask = cv2.bitwise_or(mask_brown, mask_rose)

    # apply brown and rose mask
    extracted_image = cv2.bitwise_and(frame, frame, mask=combined_mask)

    # Crop the bottom part of the image
    limit_y = math.floor(3 / 4 * extracted_image.shape[0])
    bottom_image = extracted_image[
        limit_y : extracted_image.shape[0], 0 : extracted_image.shape[1]
    ]

    # Convert the bottom image to grayscale
    gray_image = cv2.cvtColor(bottom_image, cv2.COLOR_BGR2GRAY)

    # Compute Otsu's threshold
    otsu_thresh, _ = cv2.threshold(
        gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
    )

    # Set lower and upper thresholds relative to Otsu's threshold
    lower = 0.5 * otsu_thresh
    upper = 1.5 * otsu_thresh

    # get edges
    edges = cv2.Canny(gray_image, lower, upper)

    return edges, limit_y

Get lines from edges

In [46]:
def get_lines(edges):
    # Apply Probabilistic Hough Line Transform (allow to set minLineLength and maxLineGap)
    min_line_length = 50
    max_line_gap = 10
    lines_p = cv2.HoughLinesP(
        edges,
        1,
        np.pi / 180,
        100,
        minLineLength=min_line_length,
        maxLineGap=max_line_gap,
    )
    return lines_p

Calculate the slope of the line

In [47]:
# Function to calculate the angle of a line
def calculate_angle(x1, y1, x2, y2):
    return math.degrees(math.atan2(y2 - y1, x2 - x1))

Filter horizontal lines

In [48]:
def get_horizontal(lines_p, tolerance=20):
    horizontal = []
    if lines_p is not None:
        for line in lines_p:
            x1, y1, x2, y2 = line[0]
            angle = calculate_angle(x1, y1, x2, y2)
            if abs(angle) <= tolerance:
                horizontal.append(line)
    return horizontal

Funzione di detection della linea inferiore

In [49]:
""" Detection della linea inferiore da un singolo frame"""


def bottom_detection(frame):
    # get edges
    edges, limit_y = get_edges(frame)

    # get the lines
    lines_p = get_lines(edges)

    # # Convert all lines to (rho, theta)
    # if lines_p is not None:
    #     rho_theta_lines = []
    #     for line in lines_p:
    #         x1, y1, x2, y2 = line[0]  # Extract line coordinates
    #         rho, theta = convert_to_rho_theta(x1, y1, x2, y2)
    #         rho_theta_lines.append((rho, theta))
    # else:
    #     return [0, 0]

    # if len(rho_theta_lines) == 0:
    #     # if no lines are found, return an unfeasible value (vertical line at rho = 0)
    #     return [0, 0]

    # filter horizontal lines
    horizontal_lines = get_horizontal(lines_p)

    # if rho_theta_lines is not None:
    #     horizontal_lines = [line for line in rho_theta_lines if abs(line[1] - np.pi/2) <= tolerance]

    # if len(horizontal_lines) == 0:
    #     # if no horizontal lines are found, return an unfeasible value (vertical line at rho = 0)
    #     return [0, 0]
    # else:
    #     # select the first horizontal line
    #     horizontal_line = horizontal_lines[0]

    # # Convert the found line in the reference system of the original image
    # adjusted_rho = horizontal_line[0] + limit_y * np.sin(horizontal_line[1])
    # horizontal_line = (adjusted_rho, horizontal_line[1])

    if len(horizontal_lines) == 0:
        return None

    # adjust y coordinates to come back to the original image points
    horizontal_line = horizontal_lines[0][0]
    horizontal_line[1] = horizontal_line[1] + limit_y
    horizontal_line[3] = horizontal_line[3] + limit_y

    # return the horizontal line
    return horizontal_line

Funzione per disegnare la linea sul frame

In [50]:
def write_line_on_frame(frame, line):
    # Create a copy of the original frame to draw the first line
    modified_frame = np.copy(frame)

    # Extract the first line's rho and theta
    if line:
        rho, theta = line

        # Calculate the endpoints of the line
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))

        # Draw the first line on the frame
        cv2.line(modified_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # return the modified frame
    return modified_frame

In [51]:
"""Disegna la linea sul frame"""


def write_line_on_frame_cartesian(frame, line):
    # Create a copy of the original frame to draw the first line
    modified_frame = np.copy(frame)

    if line is not None:
        x1, y1, x2, y2 = line

        # Allunga la linea di 1000 pixel da entrambe le estremità
        x1_ext, y1_ext, x2_ext, y2_ext = extend_line(x1, y1, x2, y2, length=1000)

        # Disegna la linea estesa
        cv2.line(modified_frame, (x1_ext, y1_ext), (x2_ext, y2_ext), (0, 255, 0), 2)

    # return the modified frame
    return modified_frame

Estende le linee per farle vedere meglio

In [52]:
# Funzione per estendere una linea
def extend_line(x1, y1, x2, y2, length=1000):
    # Calcola la lunghezza originale della linea
    dx, dy = x2 - x1, y2 - y1
    norm = np.sqrt(dx**2 + dy**2)  # Distanza euclidea tra i due punti

    # Evita divisioni per zero
    if norm == 0:
        return x1, y1, x2, y2

    # Calcola i punti estesi
    x1_ext = int(x1 - length * (dx / norm))
    y1_ext = int(y1 - length * (dy / norm))
    x2_ext = int(x2 + length * (dx / norm))
    y2_ext = int(y2 + length * (dy / norm))

    return x1_ext, y1_ext, x2_ext, y2_ext

Main of the code

In [53]:
video_number = "8"
# Define the relative path to the video file
notebook_dir = Path().resolve()
project_root = notebook_dir.parent.parent
video_path = (
    project_root
    / "data"
    / f"recording_{video_number}"
    / f"Recording_{video_number}.mp4"
)
video_path = str(video_path)

# Load the video
cap = cv2.VideoCapture(video_path)

# Reset the video to the beginning
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Define the codec and create a VideoWriter object to save the modified frames
output_path = (
    project_root / "data" / f"recording_{video_number}" / "Bottom_line_video.mp4"
)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Use 'mp4v' codec for MP4 format
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(str(output_path), fourcc, fps, (frame_width, frame_height))


# Initialize num_frame e horizontal_line
horizontal_lines = [None] * int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
num_frame = 0

# Loop through each frame in the video
while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video or failed to read the frame:", num_frame)
        break

    # Perform operations on the current frame
    horizontal_lines[num_frame] = bottom_detection(frame)
    if horizontal_lines[num_frame] is None:
        modified_frame = frame  # Keep the original frame if no line is detected
    else:
        # Write the modified frame to the output video
        modified_frame = write_line_on_frame_cartesian(
            frame, horizontal_lines[num_frame]
        )

    # Write the modified frame to the output video
    out.write(modified_frame)

    # Increment the frame counter
    num_frame += 1

# Release the video capture and writer objects
# cap.release()
out.release()

End of video or failed to read the frame: 119


In [54]:
def cartesian_to_polar(lines):
    rt_lines = []
    for line in lines:
        if line is None:
            rt_lines.append((0, 0))
            continue
        x1, y1, x2, y2 = line
        rho, theta = convert_to_rho_theta(x1, y1, x2, y2)
        rt_lines.append((rho, theta))
    return rt_lines

In [55]:
horizontal_lines_rt = cartesian_to_polar(horizontal_lines)
# Convert horizontal_lines to a DataFrame
horizontal_lines_df = pd.DataFrame(horizontal_lines_rt, columns=["rho", "theta"])

# Save the DataFrame to a CSV file
output_data_path = (
    project_root
    / "notebook"
    / "lane_detection"
    / "intermediate_data"
    / f"horizontal_lines_{video_number}_raw.csv"
)
horizontal_lines_df.to_csv(output_data_path, index=False)

print(f"Horizontal lines saved to {output_data_path}")

Horizontal lines saved to C:\Users\miche\OneDrive\Documenti\GitHub\bowling-analysis\notebook\lane_detection\intermediate_data\horizontal_lines_8_raw.csv


In [56]:
cap.release()